In [1]:
import json
import numpy as np
import pandas as pd
import plotly.express as px
import pandas as pd

In [2]:
import dash
from dash import dcc
from dash import html
import plotly.express as px
import pandas as pd
from dash.dependencies import Input,Output,State
import dash_daq as daq
import dash_bootstrap_components as dbc
import dash_core_components as dcc
import dash_html_components as html

<ipython-input-2-10120a1f5d6d>:9: UserWarning: 
The dash_core_components package is deprecated. Please replace
`import dash_core_components as dcc` with `from dash import dcc`
  import dash_core_components as dcc
<ipython-input-2-10120a1f5d6d>:10: UserWarning: 
The dash_html_components package is deprecated. Please replace
`import dash_html_components as html` with `from dash import html`
  import dash_html_components as html


# reading csv_file

In [3]:
oil_df = pd.read_csv('NEW_DATAFRAME_OIL.csv')
oil_price_df = pd.read_csv('oil_price.csv')

In [4]:
oil_df=oil_df.drop(['Unnamed: 0'],axis=1)
oil_price_df=oil_price_df.drop(['Unnamed: 0'],axis=1)

In [5]:
india_states = json.load(open("gz_2010_us_040_00_500k.json", "r"))

In [6]:
oil_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 52 entries, 0 to 51
Data columns (total 24 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   state   52 non-null     object 
 1   2000    52 non-null     float64
 2   2001    52 non-null     float64
 3   2002    52 non-null     float64
 4   2003    52 non-null     float64
 5   2004    52 non-null     float64
 6   2005    52 non-null     float64
 7   2006    52 non-null     float64
 8   2007    52 non-null     float64
 9   2008    52 non-null     float64
 10  2009    52 non-null     float64
 11  2010    52 non-null     float64
 12  2011    52 non-null     float64
 13  2012    52 non-null     float64
 14  2013    52 non-null     float64
 15  2014    52 non-null     float64
 16  2015    52 non-null     float64
 17  2016    52 non-null     float64
 18  2017    52 non-null     float64
 19  2018    52 non-null     float64
 20  2019    52 non-null     float64
 21  2020    52 non-null     float64
 22  id  

# Data T  oil_df

In [7]:
#changing dataframe shape to plot the barchart

oil_df_T = oil_df.T
oil_df_T.reset_index(inplace=True)
col = oil_df_T.iloc[0]
oil_df_T.columns = col

oil_df_T=oil_df_T.rename(columns={'state':'year'})


In [8]:
oil_df_T = oil_df_T.drop([0,22,23])

In [9]:
oil_df_T.head()

,year,Alaska,Arkansas,California,Colorado,Kansas,Louisiana,Montana,New Mexico,North Dakota,...,Mississippi,Missouri,Nebraska,New Hampshire,Oklahoma,Oregon,South Carolina,South Dakota,Vermont,Virginia
1,2000,458995,171642,265580,752985,525729,1.45501e+06,69936,1.6953e+06,52426,...,98526.7,97526.7,96526.7,95526.7,94526.7,93526.7,92526.7,91526.7,90526.7,89526.7
2,2001,471440,166804,266824,817206,480145,1.50209e+06,81397,1.68912e+06,54732,...,98941.3,97941.3,96941.3,95941.3,94941.3,93941.3,92941.3,91941.3,90941.3,89941.3
3,2002,463301,161871,249205,937245,454901,1.36175e+06,86075,1.63208e+06,57048,...,93068.3,92068.3,91068.3,90068.3,89068.3,88068.3,87068.3,86068.3,85068.3,84068.3
4,2003,489757,169599,226216,1.01128e+06,418893,1.3504e+06,86027,1.60402e+06,55693,...,85405.3,84405.3,83405.3,82405.3,81405.3,80405.3,79405.3,78405.3,77405.3,76405.3
5,2004,471899,187069,208919,1.07924e+06,397121,1.35325e+06,96762,1.63254e+06,55009,...,79639.7,78639.7,77639.7,76639.7,75639.7,74639.7,73639.7,72639.7,71639.7,70639.7


In [13]:
oil_df_T=oil_df_T.astype(float) 

In [14]:
#adding geo_id column to datafram

state_id_map = {}
for feature in india_states["features"]:
    feature["id"] = feature["properties"]["GEO_ID"]
    state_id_map[feature["properties"]["STATE"]] = feature["id"]
df_ = pd.DataFrame(list(state_id_map.items()),columns = ['id','geo_id']) 
df_['id']=df_['id'].astype(float)


In [15]:
m=list(oil_df['2000'].sort_values(ascending=False).head(15).index)
oil_df_copy = oil_df.iloc[m]

In [16]:
import plotly.graph_objects as go

number = go.Figure()

# Dashboard

In [17]:
app = dash.Dash(external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css',dbc.themes.DARKLY])

app.layout = html.Div(children=[
    html.H1('United States Crude Oil Production',style={'textAlign':'center','color':'gold'}),
    dash.html.Br(),
    dash.html.Br(),
    
    dcc.Slider(
            id='my_slider',
            min=oil_df_T['year'].min(),
            max=oil_df_T['year'].max(),
            value=oil_df_T['year'].min(),
            marks = {years:years for years in range(2000,2021)}),
    
    dash.html.Br(),
    
    dcc.Dropdown(
            id='drop',
            options=[{'label':str(state),'value':str(state)} for state in oil_df.state],
            value=None,
            placeholder = 'Chooce the state.....',
            style={'width': '100%','height':'40%','color':'blue'}),
    
    dash.html.Br(),
    
    html.Div([dcc.Graph(id='number',figure={})], className='row'),

    html.Div([
            html.Div([ 
            dcc.Graph(id='mygraph1',figure={})],className='six columns'),
#========================================================================

            html.Div([
#                 html.Div([dcc.Graph(id='number',figure={})], className='row'),
                html.Div([dcc.Graph(id='graph2',figure={})] ,className='row')
             
            ], className='six columns')

            ],className='row'),

#===============================================================================================

            

#=======================================================================================================================
             html.Div([
             dcc.Graph(id='mygraph2',figure={}) 
             ], className='row'),
#======================================================

#==================================================================================================


])
#===============================================================================================
@app.callback(
    Output(component_id = 'mygraph1',component_property='figure'),
    Output(component_id = 'number',component_property='figure'),
    Output(component_id = 'graph2',component_property='figure'),
    Output(component_id = 'mygraph2',component_property='figure'),
    Input(component_id = 'my_slider',component_property='value'),
    Input(component_id = 'drop',component_property='value'))


def graph(my_slider,dropdown):

    mapp = px.choropleth_mapbox(
    oil_df,
    locations="geo_id",
    geojson=india_states,
    color=str(my_slider),
    hover_name="state",
    title="U.S. Crude Oil Production BBl",
    mapbox_style="carto-positron",
    center={"lat": 24, "lon": 78},
    zoom=3,
    opacity=0.5,
    width=1750,
    height=700,template='plotly_dark')
    
    
    fig =px.line(oil_df_T,y=dropdown,x='year',width=800,height=600,template='plotly_dark')
    fig2 =px.bar(oil_df_copy,y='state',x=oil_df_copy[str(my_slider)],width=800,height=600,template='plotly_dark')
    
    number = go.Figure()

    number.add_trace(go.Indicator(
    mode = "number",
    number ={'suffix':"$"},
    value = oil_price_df.loc[my_slider - 2000][0],
    title='Oil Price',
    domain = {'x': [0, 0.3], 'y': [0, 0]}))

    fig.update_traces(mode='lines+markers')

    number.add_trace(go.Indicator(title='Revenue in the United States',
    mode = "number",
    number ={'suffix':" billion $"},
    value = round(oil_df_T.loc[my_slider - 1999].sum()*(oil_price_df.loc[my_slider - 2000][0])/1000000000,3),
    domain = {'x': [0.5, 1], 'y': [0, 1]}))
    number.update_layout(
    autosize=False,
    width=1800,height=300
    ,font_color='gold', paper_bgcolor="rgb(34,34,34)")
    
    return fig,number,fig2,mapp



In [ ]:
app.run_server()

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [06/Nov/2021 10:51:59] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [06/Nov/2021 10:52:01] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [06/Nov/2021 10:52:01] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [06/Nov/2021 10:52:02] "GET /_dash-component-suites/dash/dcc/async-slider.js HTTP/1.1" 200 -
127.0.0.1 - - [06/Nov/2021 10:52:02] "GET /_dash-component-suites/dash/dcc/async-dropdown.js HTTP/1.1" 200 -
127.0.0.1 - - [06/Nov/2021 10:52:02] "GET /_dash-component-suites/dash/dcc/async-graph.js HTTP/1.1" 200 -
127.0.0.1 - - [06/Nov/2021 10:52:02] "GET /_dash-component-suites/dash/dcc/async-plotlyjs.js HTTP/1.1" 200 -
127.0.0.1 - - [06/Nov/2021 10:52:08] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [06/Nov/2021 14:48:12] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [06/Nov/2021 14:48:17] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [06/Nov/2021 14:49:31] "POST /_dash-